# Введение в искусственные нейронные сети
# Урок 3. Keras

# Рабочее место пользователя KERAS

### Я постарался собрать основные пакеты, которые могут понадобиться при установке  KERAS на новом рабочем месте

In [13]:
# pip install mnist

In [ ]:
# pip install tensorflow

In [ ]:
# conda uninstall keras
# pip install keras

In [ ]:
# pip install pydot

In [ ]:
# pip install graphviz

In [ ]:
# pip install h5py

In [5]:
import h5py
h5py.run_tests()

...........s...........................................x...........................s............................................x....................................s/Users/radik/opt/anaconda3/lib/python3.7/site-packages/h5py/_hl/files.py:258: H5pyDeprecationWarning: File.fid has been deprecated. Use File.id instead.
  "Use File.id instead.", H5pyDeprecationWarning)
...s......ss.......................................................................................................ssssss..................................E..................................x....x.........................x......x..................................................ssss...................s........................................
ERROR: test_deprecation_available_ftypes (h5py.tests.old.test_h5t.TestDeprecation)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/radik/opt/anaconda3/lib/python3.7/site-packages/h5py/tests/old/test_h5t.py", line 2

<unittest.runner.TextTestResult run=509 errors=1 failures=0>

In [1]:
import pydot
pydot.graph_from_dot_file()

### К-сожалению, мне не удалось понять философию, того как правильно применять h5py и pydot. А других способов визуализации графов модели NN мне неизвестно

# Датасет  MNIST

In [ ]:
# The full neural network code!
###############################
import numpy as np
import mnist

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.callbacks import CSVLogger

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) # - 0.5
test_images = (test_images / 255)   #- 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

# Build the model.
model = Sequential([
  Dense(10, activation='relu', input_shape=(784,)),
  Dense(20, activation='softmax'),
  Dense(30, activation='relu'),
  Dense(10, activation='softmax'),
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

csv_logger = CSVLogger('training.log', separator='_',append=True)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=500,
  batch_size=32,
  callbacks=[csv_logger],
)

# Evaluate the model.
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

# Save the model to disk.
model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 25 test images.
predictions = model.predict(test_images[:25])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) 

# Check our predictions against the ground truths.
print(test_labels[:25]) 

In [ ]:
Epoch 499/500
60000/60000 [==============================] - 3s 43us/step - loss: 0.0616 - accuracy: 0.9813
Epoch 500/500
60000/60000 [==============================] - 3s 43us/step - loss: 0.0602 - accuracy: 0.9817
10000/10000 [==============================] - 0s 34us/step
[7 2 1 0 4 1 4 9 9 9 0 6 9 0 1 5 9 7 8 4 9 6 6 5 4]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4]

In [ ]:
# Build the model ver2.
model = Sequential([
  Dense(10, activation='relu', input_shape=(784,)),
  Dense(30, activation='softmax'),
  Dense(30, activation='softmax'),
  Dense(30, activation='softmax'),
  Dense(10, activation='softmax'),
])

60000/60000 [==============================] - 3s 57us/step - loss: 0.1583 - accuracy: 0.9564
10000/10000 [==============================] - 0s 33us/step 
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1]

## Вывод:
### Я решил не гоняться за показателем accuracy, а пойти от упрощения.
Модель из 2-х слоев по 10 элементов показывала хорошие результаты accuracy = 0.94. 
Но 8 и 19-й элемент корректно не распозновались: вместо 5 определялась 6 (9), вместо 3 - 8. 
### Усложнение модели за счет увеличение кол-ва эпох, слоев и элементов в них повысила accuracy на 6%, но 2 элемента по-прежнему идентифицируются неверно. 
Можно списать это на качество датасета, либо на предел NN на данной элементной базе; уверен, есть такая конфигурация при которой и эти элементы корректно распознаются

# Датасет Boston Housing

### Я решил применить NN для предсказания линейной регрессии в популярном датасете Boston Housing 

In [ ]:
import numpy as np
import mnist

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.wrappers.scikit_learn import KerasRegressor

from keras.callbacks import CSVLogger
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=(x_train.shape[1]), init='normal', activation='relu'))
    model.add(Dense(20, init='normal', activation='linear'))  # ten neuron, linear activation
    model.add(Dense(1, init='normal', activation='linear'))  # one neuron, linear activation
    # Compile model
    model.compile(loss='mse', optimizer='adam')  # mse loss
    return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=5000, batch_size=20, verbose=2)  # KerasRegressor for regression problem
estimator.fit(x_test, y_test)

csv_logger = CSVLogger('boston_housing.log', separator='_',append=True)

# Save the model to disk.
model.save_weights('boston_housing.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
predictions = estimator.predict(x_test[:5])

# Print our model's predictions.
print(predictions) # 

# Check our predictions against the ground truths.
print(y_test[:5]) # 

In [ ]:
Epoch 4999/5000
 - 0s - loss: 2.9452
Epoch 5000/5000
 - 0s - loss: 3.6982
[ 4.7438025 15.9756    19.063217  28.179684  21.890736 ]
[ 7.2       18.8       19.        27.        22.2]

## Вывод:
### для получения устраивающего меня результата пришлось:
1. отказаться от метрики accuracy в сторону mse, т.к. из-за ее применения при работе с ЛР возникают проблемы
2. переключиться с KerasClassifier на KerasRegressor, т.к. мы имеем дело с ЛР. Хотя результат был, но очень низкий
3. использовать метод активации linear на внутренних слоях, т.к. он предназначен для применения в ЛР, ReLU - здесь не место. А также я добавил большее кол-во слоев, но не в ущерб временным затратам и используемой ОЗУ (на моем ноутбуке всего 8 ГБ ОЗУ и 4 потока)
4. для обучения модели понадобилось на порядок больше эпох, но итоговый результат того стоил. Кажется логичным, что для ЛР требуется более глубокое обучение, по сравнению с обычной классификацией

# TIPS & TRICKS

### CSVLogger
#### Мне всегда удобно работать с внешним лог-файлам. Поэтому этот фреймворк не стал для меня исключением 

In [ ]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('training.log')

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=5,
  batch_size=32,
  callbacks=[filename=csv_logger, separator='+', append=True],
)